Site Zhihu : https://www.zhihu.com/

---

En résumé

    Installez et configurez Selenium dans Colab.

    Connectez-vous manuellement à Zhihu via QR code.

    Définissez les fonctions utilitaires (parsing date, extraction heuristique).

    Lancez une boucle sur mots-clés 热词/潮词/新词 : récupérer URLs, visiter, extraire date ≥ 2015, lister néologismes et définitions heuristiques.

    Construisez un DataFrame pandas, dédupliquez, filtrez à environ 200 lignes pertinentes.

    Exportez en CSV et téléchargez.

    Vérifiez et complétez manuellement les définitions manquantes.


In [3]:
# Installer Chromium, ChromeDriver, Selenium, pandas, bs4, dateutil
!apt-get update -qq
!apt-get install -y -qq chromium-browser chromium-chromedriver
!pip install -q selenium pandas python-dateutil bs4


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [7]:
!pip install -q webdriver-manager

In [10]:
!apt-get update
!apt-get install -y chromium-browser


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [11]:
chrome_options.binary_location = "/usr/bin/chromium-browser"


In [18]:

# 然后配置 Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

def init_driver():
    # 创建 les options pour Chrome
    chrome_options = Options()
    # Mode headless nouvelle version
    chrome_options.add_argument('--headless=new')
    # Pour éviter les problèmes de sandbox dans Colab
    chrome_options.add_argument('--no-sandbox')
    # Réduire l’usage de la mémoire partagée
    chrome_options.add_argument('--disable-dev-shm-usage')
    # Désactivation du GPU (pour compatibilité)
    chrome_options.add_argument('--disable-gpu')
    # Port pour le debugging, parfois utile pour éviter DevToolsActivePort error
    chrome_options.add_argument('--remote-debugging-port=9222')
    # User-Agent standard pour éviter d’être bloqué
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"
    )
    # Indiquer le chemin vers l’exécutable Chromium installé
    chrome_options.binary_location = '/usr/bin/chromium-browser'

    # Spécifier le Service pointant vers chromedriver installé
    service = Service('/usr/bin/chromedriver')

    # Lancer le driver
    driver = webdriver.Chrome(service=service, options=chrome_options)
    # Facultatif : taille de fenêtre pour certains besoins de rendu
    driver.set_window_size(1200, 800)
    return driver

# Tester l'initialisation
driver = init_driver()
print("ChromeDriver démarré avec succès")
# N’oubliez pas de driver.quit() après usage


WebDriverException: Message: Service /usr/bin/chromedriver unexpectedly exited. Status code was: 1


In [15]:
#Connexion Zhihu via QR code (manuelle)

import time
from IPython.display import Image, display
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

def login_via_qr(driver, timeout=120):
    driver.get("https://www.zhihu.com/signin?next=%2F")
    time.sleep(2)
    path="/mnt/data/zhihu_qr.png"
    driver.save_screenshot(path)
    display(Image(path))
    print(f"→ Scannez le QR code affiché dans les {timeout}s avec l’app Zhihu.")
    start=time.time()
    while True:
        time.sleep(3)
        if "signin" not in driver.current_url:
            print("Connexion détectée, URL:", driver.current_url)
            break
        # Optionnel: détecter avatar connecté
        if time.time()-start>timeout:
            raise TimeoutError("QR non scanné dans le temps imparti.")
    time.sleep(2)

# Lancer la connexion
login_via_qr(driver)


NameError: name 'driver' is not defined

In [ ]:
#3. Fonctions utilitaires pour dates et extraction

import re
from dateutil import parser as date_parser
from datetime import datetime

def is_date_after_2015(date_str):
    """
    Parse date_str (ISO, 'YYYY-MM-DD', ou 'YYYY年M月D日') et retourne (date_obj, True/False).
    """
    if not date_str:
        return None, False
    s=date_str.strip()
    date_obj=None
    try:
        date_obj = date_parser.isoparse(s)
    except:
        pass
    if date_obj is None:
        m=re.match(r'(\d{4})年\s*(\d{1,2})月\s*(\d{1,2})日', s)
        if m:
            y,mo,d = m.groups()
            try:
                date_obj = datetime(int(y), int(mo), int(d))
            except:
                pass
    if date_obj is None:
        try:
            date_obj = date_parser.parse(s, fuzzy=True)
        except:
            pass
    if date_obj is None:
        return None, False
    return date_obj, (date_obj.year >= 2015)

def extract_definition(text, word):
    """
    Heuristique pour extraire définition de `word` dans `text`.
    Cherche constructions “XXX是指...”, “XXX指的是...”, “XXX意为...”.
    """
    patterns=[
        rf'“{re.escape(word)}”\s*(?:是指|指的是|指)\s*([^。；\n]+)',
        rf'{re.escape(word)}\s*(?:是指|指的是|指)\s*([^。；\n]+)',
        rf'“{re.escape(word)}”.*?意为\s*([^。；\n]+)',
        rf'{re.escape(word)}.*?意为\s*([^。；\n]+)'
    ]
    for pat in patterns:
        m=re.search(pat, text)
        if m:
            return m.group(1).strip()
    return ""


In [ ]:
#4. Recherche de URLs liées aux mots-clés
from selenium.webdriver.common.by import By

def search_and_collect_urls(driver, keyword, max_scrolls=5):
    """
    Recherche Zhihu pour keyword, retourne une liste d'URLs pertinentes (questions, articles).
    """
    search_url = f"https://www.zhihu.com/search?type=content&q={keyword}"
    driver.get(search_url)
    time.sleep(3)
    for _ in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    elems = driver.find_elements(By.CSS_SELECTOR, "a")
    urls=set()
    for a in elems:
        href=a.get_attribute("href") or ""
        if any(p in href for p in ["zhihu.com/question/", "zhuanlan.zhihu.com/p/", "zhihu.com/p/"]):
            if "signin" in href: continue
            urls.add(href.split('?')[0])
    return list(urls)


In [ ]:
#5. Extraction de néologismes et définition dans chaque page

from selenium.common.exceptions import NoSuchElementException

def extract_neologisms_from_text(text):
    """
    Heuristique: repérer listes après '新词', '热词', '潮词' :
    cherche séquences de mots séparés par '、' ou virgules dans la même phrase.
    Retourne liste de mots candidates.
    """
    neos=set()
    # Cherche pattern '新词：...'
    patterns = [
        r'新词[：:]\s*([^。；\n]+)',
        r'热词[：:]\s*([^。；\n]+)',
        r'潮词[：:]\s*([^。；\n]+)',
    ]
    for pat in patterns:
        for m in re.findall(pat, text):
            seg = m.strip()
            # Séparer par délimiteurs courants
            parts = re.split(r'[、,，;；]', seg)
            for w in parts:
                w=w.strip()
                # garder mots courts (2-6 caractères) ou selon heuristique
                if 1 < len(w) <= 6:
                    neos.add(w)
    return list(neos)

def parse_page_and_extract(driver, url):
    """
    Ouvre url, extrait date, puis liste de néologismes + définitions heuristiques.
    Retourne liste d'entrées dict {néologisme, signification, source, date}, ou [].
    """
    try:
        driver.get(url)
        time.sleep(2)
    except Exception as e:
        print("Erreur ouverture", url, e)
        return []
    # Extraire date
    date_str=""
    try:
        t=driver.find_element(By.TAG_NAME, "time")
        date_str = t.get_attribute("datetime") or t.text
    except NoSuchElementException:
        pass
    date_obj, ok = is_date_after_2015(date_str)
    if not ok:
        return []
    page_text=""
    try:
        el=driver.find_element(By.CSS_SELECTOR, "div.RichContent-inner")
        page_text=el.text
    except NoSuchElementException:
        try:
            page_text=driver.find_element(By.TAG_NAME, "body").text
        except:
            page_text=""
    neos = extract_neologisms_from_text(page_text)
    entries=[]
    for neo in neos:
        definition = extract_definition(page_text, neo)
        entries.append({
            "néologisme": neo,
            "signification": definition,
            "source": url,
            "date": date_obj.date().isoformat()
        })
    return entries


In [ ]:
#6. Boucle principale et constitution du DataFrame

import pandas as pd

keywords = ["热词", "潮词", "新词"]
all_entries=[]
seen_urls=set()

for kw in keywords:
    print(f"==== Recherche pour '{kw}' ====")
    urls = search_and_collect_urls(driver, kw, max_scrolls=5)
    print(f" trouvées ~{len(urls)} URLs")
    for url in urls:
        if url in seen_urls: continue
        seen_urls.add(url)
        ents = parse_page_and_extract(driver, url)
        if ents:
            all_entries.extend(ents)
        if len(all_entries) >= 300:  # tampon, on vise ~200 valides après dédup
            break
    if len(all_entries) >= 300:
        break

# Construire DataFrame
df = pd.DataFrame(all_entries)
# Dédupliquer sur (néologisme, source)
df.drop_duplicates(subset=["néologisme","source"], inplace=True)
# Si plus de 200, on peut trier par date (desc) ou garder aléatoire
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.sort_values('date', ascending=False).head(200)
df.reset_index(drop=True, inplace=True)

print("Total extraits:", len(df))
display(df.head(10))

# Export CSV
csv_path="/mnt/data/neologismes_zhihu.csv"
df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print("CSV enregistré:", csv_path)


In [ ]:
#7. Téléchargement du CSV
from google.colab import files
files.download("/mnt/data/neologismes_zhihu.csv")


In [ ]:
#Exemple d’amélioration de l’extraction de listes (<li>)

def parse_page_and_extract(driver, url):
    # ... ouverture et date comme avant ...
    # Récupérer page_text et DOM:
    try:
        container = driver.find_element(By.CSS_SELECTOR, "div.RichContent-inner")
    except NoSuchElementException:
        container = None
    page_text = container.text if container else driver.find_element(By.TAG_NAME,"body").text

    # Extraire néologismes via texte
    neos = set(extract_neologisms_from_text(page_text))
    # Extraire néologismes via <li>
    if container:
        try:
            lis = container.find_elements(By.CSS_SELECTOR, "ul li")
            for li in lis:
                txt = li.text.strip()
                # si format “1. 躺平：……” ou “躺平——放弃…”
                # récupérer la partie avant : ou — ou 空格
                w = txt.split('：')[0].split('—')[0].split('—')[0].split()[0]
                if 1 < len(w) <= 6:
                    neos.add(w)
        except:
            pass
    entries=[]
    for neo in neos:
        definition = extract_definition(page_text, neo)
        entries.append({
            "néologisme": neo,
            "signification": definition,
            "source": url,
            "date": date_obj.date().isoformat()
        })
    return entries
